**I. STRATEGY DESIGN**

**task 2.1**

Let $mid_t$, $bid_t$, $ask_t$, $spread_t$ $-$ be the values of the mid-price, bid, ask and spread at time $t$.

Let $spread_{base}$ be the base spread.
At the beginning I would choose $spread_{base}$ slightly tighter than the average spread on market in order to stay competitive.

Let $vol_t$ and $vol_{target}$ be the current volatility at time $t$ and the target volatility level.
And choose $vol_{target}$  as the median (or average, that I think worst) short-term volatility over a recent historical window. Probably something like the median 10-minute volatility over the last few days.

Let $\alpha$ be the coefficient that controls how fast the spread adapts to changes in volatility.
I would choose $\alpha$ as $0.2$ $-$ small enough to avoid overreacting to temporary spikes in volatility. But at the same time it has large impact to make wider spread when volatility is above the target level.

$bid_t = mid_t - \frac{spread_t}{2}$

$ask_t = mid_t + \frac{spread_t}{2}$

$spread_t = spread_{base} \cdot (1 + \alpha \cdot \frac{vol_t}{vol_{target}})$

**task 2.2**

For maximum inventory limit $Q_{max}$ I would choose a relatively conservative limit.
Hope that this helps avoid being stuck with a large one-sided position when flow becomes directional.


When inventory becomes significantly positive or negative, I skew my quotes:

 - If I am long, I shift the mid slightly down, make my ask more aggressive, and move the bid further from mid.

 - If I am short, I shift the mid upwards, make the bid more aggressive, and widen the ask.

 - Can use some math model similar to block with SOME THEORY

This way helps my model gradually reduce inventory without stopping quoting completely.


I will pause qouting (probably only on one side) when:
 
 - inventory is too close to $Q_{max}$

 - after a strong directional move that goes against my current inventory

Those the main criterias for stoping quoting, also I can consider impact of thin order book, sudden spikes of volatility and many others.

**task 2.3**

I want to use two thresholds as **material** price moves:
1. mid-price changeв by more than 0.5%
2. move is larger than 10-minute volatility unit

When I will refresh:
1. When at least one material price move occurs
2. If volatility increases or market becomes more competitive I need to adjust my spreads
3. After a large fill I need to refresh quotes agresively to better adapt to market change 

**II. SOME THEORY**

**Flow and thin-book adjustment**

Estimate of net taker flow over a short recent window:

$net\_flow_t = taker\_buy_t - taker\_sell_t$, then:

If net_flow is strongly positive (persistent taker buys), so can be expected upward pressure on price. 
MM can do:

 - slightly move the mid upwards when quoting, and/or
 
 - widen the spread to protect the ask side.

If net_flow is strongly negative (persistent taker sells), MM can do the symmetric adjustment.

**INVENTORY CONTROL**

An important component of a market-making model is inventory control — preventing the market maker from accumulating positions that are too large relative to risk tolerance.

**Maximum inventory**

Let $Q_t$ be the current inventory.
MM defines a maximum allowed inventory $Q_{max}$ than $∣Q_t∣ \le Q_{max}$.

The value of $Q_{max}$ typically depends on account size, volatility, and liquidity.
Once the inventory approaches this boundary, the MM reduces quoting activity on the side that increases the position.


**Rebalancing when inventory becomes skewed**

When the inventory is leaning too far to one side, the MM “skews” their quotes to encourage trades that reduce the imbalance.
For example:

If $Q_t > 0$ (MM is long):

 - make the ask more aggressive (closer to mid),

 - make the bid less aggressive (further from mid).

If $Q_t < 0$ (MM is short):

 - bring the bid closer,

 - move the ask further away.

A simple rule is to shift the effective mid-price according to inventory pressure:

$mid_t^{inv} = mid_t - \beta \cdot \frac{Q_{t}}{Q_{max}}$​


where $\beta$ controls how strongly quotes are adjusted based on inventory.

**INFO FOR PRICE REFRESH**

A "material" price move is a large enough change in the mid-price such that the current bid/ask quotes no longer correspond to fair value.

This creates a risk that informed traders will exploit old quotes. For example, if the price moves up, MMs ask may become very cheap relative to new fair price (and similarly when price drops). Because of it MMs have to decide whether to refresh or keep their quotes.

It may seem that refreshing immediately is always the best option, but this is **not true**:

1. This can be just a noisy price change, so if the MM tries to update prices every time, they end up constantly reacting to tiny fluctuations around the fair price that could simply be kept.

2. Constant small updates create useless technical noise, so churn increases and it does not bring any improvement.

3. If price moves chaoticaly, informed traders may exploit situations when one of the quotes gets delayed or temporarily becomes stale.

4. So frequent small updates also bring additional artificial volatility.

**What can be considered for MM when choose whether to update or keep:** 

1. Update if some move passes some threshold. For example, change of more than 0.5% of ptice

2. Update in situation when there appear risk of adverse selection

3. Overall MM may update price to stay more competitive and be close to the mid

4. MM should keep quotes in case of small noisy price changes

5. Quotes can be kept if market has low volatility of if price tend to revert back